In [79]:
import pandas as pd
import numpy as np

#For parsing
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
import re
import json

In [15]:
urls_df = pd.read_csv("URL_list.csv")
urls_df.head()

,max(page)
0,https://www.factorybuys.com.au/products/euro-t...
1,https://dunlin.com.au/products/beadlight-cirrus
2,https://themodern.net.au/products/hamar-plant-...
3,https://furniturefetish.com.au/products/oslo-o...
4,https://hemisphereliving.com.au/products/


In [16]:
url_series = pd.Series(urls_df["max(page)"]) #To make code easier
url_series

,max(page)
0,https://www.factorybuys.com.au/products/euro-t...
1,https://dunlin.com.au/products/beadlight-cirrus
2,https://themodern.net.au/products/hamar-plant-...
3,https://furniturefetish.com.au/products/oslo-o...
4,https://hemisphereliving.com.au/products/
...,...
699,https://signaturefinefurniture.ca/products/jac...
700,http://aonefurniture.in/userdata/products/no%2...
701,https://furnituremama.com/products/three-door-...
702,https://gfurniture.ca/products/


# Предобработка данных
Сохраним только открывающиеся ссылки

In [92]:
def check_is_url_parsing(url):
  headers = {"User-Agent": "Mozilla/5.0"}
  try:
      response = requests.get(url, headers=headers, timeout=10)
      response.raise_for_status() #check for html error
      soup = BeautifulSoup(response.text, "html.parser")

      return True

  except requests.exceptions.RequestException as e:
    print(f"Attemp to URL falied: {e}")
    return False

In [96]:
def create_good_urls_csv():
  """Создает файл 'ParsingURL_list.csv' с посещаемыми ссылками"""
  good_urls =[]
  for url in url_series:
    if(check_is_url_parsing(url)):
      good_urls.append(url)

  urls = pd.Series(good_urls, name="url")
  urls.to_csv("ParsingURL_list.csv")

In [95]:
# create_good_urls_csv()

#Parsing data from sites

In [80]:
def get_parsed_data(url):
    """Получает данные о продукте из JSON-LD разметки страницы"""
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        scripts = soup.find_all("script", type="application/ld+json")
        product = None

        for script in scripts:
            try:
                data = json.loads(script.string)

                if isinstance(data, list):
                    for obj in data:
                        if isinstance(obj, dict) and obj.get("@type") == "Product":
                            product = obj
                            break
                elif isinstance(data, dict) and data.get("@type") == "Product":
                    product = data

                if product:
                    return {
                        "url": url,
                        "title": product.get("name"),
                        "description": product.get("description"),
                        "price": product.get("price"),
                        "currency": product.get("priceCurrency"),
                        "rating": product.get("aggregateRating", {}).get("ratingValue"),
                        "brand": product.get("brand", {}).get("name")
                    }

            except json.JSONDecodeError:
                continue

        print(f'No product data found in JSON-LD: <a href="{url}">{quote(url)}</a>')
        return None

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

In [81]:
columns = ["url","title","description","price","currency","rating","brand"]
urls = pd.DataFrame(columns=columns)

all_data =[]
for url in url_series:
  product_data = get_parsed_data(url)

  if(product_data):
    product_data["url"] = url
    all_data.append(product_data)

# Создаем датафрейм разом
urls_df = pd.DataFrame(all_data)


Unexpected error: 'str' object has no attribute 'get'
No product data found in JSON-LD: <a href="https://furniturefetish.com.au/products/oslo-office-chair-white">https%3A//furniturefetish.com.au/products/oslo-office-chair-white</a>
Request failed: 404 Client Error: Not Found for url: https://hemisphereliving.com.au/products/
Request failed: HTTPSConnectionPool(host='home-buy.com.au', port=443): Max retries exceeded with url: /products/bridger-pendant-larger-lamp-metal-brass (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fdc6e586390>: Failed to resolve 'home-buy.com.au' ([Errno -2] Name or service not known)"))
No product data found in JSON-LD: <a href="https://interiorsonline.com.au/products/interiors-online-gift-card">https%3A//interiorsonline.com.au/products/interiors-online-gift-card</a>
Request failed: HTTPSConnectionPool(host='beckurbanfurniture.com.au', port=443): Max retries exceeded with url: /products/page/2/ (Caused by NameResolutionError("<u

In [82]:
urls_df

,url,title,description,price,currency,rating,brand
0,https://www.factorybuys.com.au/products/euro-t...,Factory Buys 32cm Euro Top Mattress - King,Pamper yourself with Factory Buys Euro Top Mat...,None,None,4.9,Factory Buys
1,https://dunlin.com.au/products/beadlight-cirrus,Beadlight Cirrus LED Reading Light,"Handmade by Beadlight in the UK, the Cirrus co...",None,None,None,Beadlight
2,https://vastinterior.com.au/products/samson-da...,Samson Daybed Single,\nNote: This is a frame only. Does not come wi...,None,None,None,Samson
3,https://www.perchfurniture.com/products/hoyt-c...,Hoyt Chair,"Designed with simplicity in mind, the Hoyt Cha...",None,None,None,Perch Furniture
4,https://claytongrayhome.com/products/palecek-c...,Coco Ruffle Mirror,Palecek presents the Coco Ruffle Mirror made o...,None,None,None,Palecek
...,...,...,...,...,...,...,...
65,https://donar.si/products/collodi,Collodi,Collodi Chair is a response to ever growing pr...,None,None,None,DONAR shop
66,https://pepegarden.co.uk/products/erica-park-b...,Erica Park Bench,Relax on our beautifully designed Erica wooden...,None,None,None,Pepe
67,https://www.theurbaneditions.com/products/mars...,Marston Wide Console Desk on Minimalist Square...,The Marston wide console desk is a streamlined...,None,None,5.0,The Urban Editions
68,https://www.nabisottomanfurniture.co.uk/produc...,CHESTERFIELD DIAMANTE BED FRAME CRUSHED VELVET...,CHESTERFIELD DIAMANTE BED FRAMECRUSHED VELVET...,None,None,None,Unbranded
